In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [9]:
# packages/libraries
import pandas as pd
import numpy as np
import os
import json
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
import ast
import cv2
import json
import os
import pandas as pd
import requests
import ast
from IPython.display import Image, display
from io import StringIO
from IPython.display import Image, display
from requests.compat import urljoin as urlj

with open('/home/magali/ct_classifier/inspect_json/model_sex/ModelOutput_SexBasic.json', 'r') as json_file: 
    data = json.load(json_file) #path to json model output

with open('/home/magali/CV4Ecology-summer-school/FinalDataset/SubsetAgeModelCocoVal_croppedID.json', 'r') as json_file: 
    raw_data = json.load(json_file) #path to json cropped image file with image path of the val dataset

In [10]:
# we'll also import the raw data to compare the images and predictions against, to make sure the data are labelled correctly:
#path = "/home/michael/pred-detector/detection-data/train-val-data_2023-08-15.csv"

# import csv data
#raw_data = pd.read_csv(path)
#raw_data.head()

# List to hold the new ids
ids = []

# Function to remove first parts of the string
def remove_prefix(filepath, prefix):
    if filepath.startswith(prefix):
        return filepath[len(prefix):]
    return filepath

# Define the prefix to remove
prefix_to_remove = '/home/magali/CV4Ecology-summer-school/PrototypeCroppedImages/PrototypeCroppedImages_Sex_Val/'

# Remove prefix and create new id column
for filepath in data['filepaths']:
    current_id = remove_prefix(filepath, prefix_to_remove)
    ids.append(current_id)

# Assign the new ids list to the data dictionary
data['id'] = ids


#id = []

# Function to remove first parts of the string
#def remove_prefix(filepath, prefix):
#    #print(type(filepath), type(prefix))
#    if filepath.startswith(prefix):
#        return filepath[len(prefix):]
#    return filepath

#data['id'] = id

# Define the prefix to remove
#prefix_to_remove = '/home/magali/CV4Ecology-summer-school/PrototypeCroppedImages/PrototypeCroppedImages_Sex_Val'

#remove prefix and create new id column
#for filepath in data['filepaths']:
#    id = remove_prefix(filepath, prefix_to_remove)
#    id.append(id)

#for i in range(len(data['filepaths'])):
#    data['filepaths'][i] = remove_prefix(data['filepaths'][i], prefix_to_remove)

# generate new id column
#for i in range(len(data['filepaths'])):
#    data['id'] = i  # or any other logic to generate unique IDs.

# Apply the function to the 'filepaths' column
#raw_data['new_filepath'] = raw_data['new_filepath'].apply(remove_prefix, prefix=prefix_to_remove)
#raw_data['new_filepath'] = '/mnt/ssd-cluster/procko/Sampled Images/' + raw_data['new_filepath']

In [11]:
#@title Load images with bounding boxes
def display_image(file_path, width=1000, height=None):
    print(file_path)
    if os.path.exists(file_path):
        display(Image(file_path, width=width, height=height))
    else:
        print('File not found')

def draw_bboxs(img_path, coordinates, out_path=None, display_now=True):
    image = cv2.imread(img_path)
    height, width, channels = image.shape
    for c in coordinates:
        start_point = (int(c[0]*width), int(c[1]*height))
        end_point = (int(c[0]*width) + int(c[2]*width), int(c[1]*height) + int(c[3]*height))
        cv2.rectangle(image, start_point, end_point, color=(0, 255, 0), thickness=2)
    if not out_path:
        out_path = img_path.replace('.', '_bboxes.')
    out_path = out_path + '.jpg'  # Specify the file extension
    cv2.imwrite(out_path, image)
    if display_now:
        display_image(out_path)
    return out_path

def select_by_species(df, species):
    return df[df['scientificName'] == species]

def show_random_species(df, species, idx=None, display_now=True):
    sdf = select_by_species(df, species)
    if sdf.empty:
        print('Species not in available photo set')
        return None
    row = sdf.sample(n=idx)
    print(row.to_markdown())
    row = row.squeeze()
    project_id = row['projectID']
    deployment_id = row['deploymentID'] #add here one row with folder path (column) when several projects
    file_name = row['fileName']
    photo_path = os.path.join('/home/magali/CV4Ecology-summer-school/PrototypeImages/PrototypeCV4E_SexModelImages', project_id, deployment_id, file_name) #add here project folder
    if not os.path.exists(photo_path):
        print('Image not found')
        return None
    row['local_path'] = photo_path
    row['bbox'] = ast.literal_eval(row['bbox'])
    draw_bboxs(photo_path, row['bbox'], display_now=display_now)
    return row

In [12]:
# Create a list to store the match categories
match_categories = []

# Iterate through the true and predicted values
for true_val, pred_val in zip(data['true_labels'], data['pred_labels']):
    if true_val == 0 and pred_val == 0:
        match_categories.append('true_negative')
    elif true_val == 1 and pred_val == 0:
        match_categories.append('false_negative')
    elif true_val == 0 and pred_val == 1:
        match_categories.append('false_positive')
    else:
        match_categories.append('true_positive')

# Add the match categories list to the dictionary
data['match_category'] = match_categories

# pick a random number
n = 18

# view an example of the updated dictionary
print("This is a", data['match_category'][n], 'where', data['pred_labels'][n], "was the prediction,",
data['true_labels'][n], 'was the true value, and the score values were', data['scores'][n])

# show the image
img = data['filepaths'][n]
pil_im = Image.open(img)
display(pil_im)

#raw_data[raw_data['new_filepath'] == data['filepaths'][n]]

This is a true_negative where 0 was the prediction, 0 was the true value, and the score values were [0.7166692018508911, 0.2833308279514313]


AttributeError: type object 'Image' has no attribute 'open'

In [13]:
# create false negatives dictionary:
false_negatives = []
for category, scores, filepaths, id in zip(data['match_category'], data['scores'], data['filepaths'], data['id']):
    if category == 'false_negative':
        false_negatives.append({'match_category': category, 'scores': scores, 'filepaths': filepaths, 'id': id})

# # sort by highest false confidence:
sorted_false_negatives = sorted(false_negatives, key=lambda x: x['scores'][0], reverse=True)
top50_false_negatives = sorted_false_negatives[0:50]


# create false positives dictionary:
false_positives = []
for category, scores, filepaths, id in zip(data['match_category'], data['scores'], data['filepaths'], data['id']):
    if category == 'false_positive':
        false_positives.append({'match_category': category, 'scores': scores, 'filepaths': filepaths, 'id': id})

# # sort by highest false confidence:
sorted_false_positives = sorted(false_positives, key=lambda x: x['scores'][0], reverse=True)
top50_false_positives = sorted_false_positives[0:50]


# # create true positives dictionary:
true_positives = []
for category, scores, filepaths, id in zip(data['match_category'], data['scores'], data['filepaths'], data['id']):
    if category == 'true_positive':
        true_positives.append({'match_category': category, 'scores': scores, 'filepaths': filepaths, 'id': id})

# # sort by highest true confidence:
sorted_true_positives = sorted(true_positives, key=lambda x: x['scores'][1], reverse=True)
top50_true_positives = sorted_true_positives[0:50]


# # create true negatives dictionary:
true_negatives = []
for category, scores, filepaths, id in zip(data['match_category'], data['scores'], data['filepaths'], data['id']):
    if category == 'true_negative':
        true_negatives.append({'match_category': category, 'scores': scores, 'filepaths': filepaths, 'id':id})

# sort by highest true confidence:
sorted_true_negatives = sorted(true_negatives, key=lambda x: x['scores'][0], reverse=True)
top50_true_negatives = sorted_true_negatives[0:50]


In [ ]:
# Display images using the filepaths key
plt.figure(figsize=(40, 20))
for idx, entry in enumerate(top50_false_negatives):
    img = Image.open(entry['filepaths'])
    plt.subplot(5,10,idx+1)
    plt.imshow(img, aspect='equal')
    plt.title(f"{entry['id']}")
    plt.axis('off')
plt.tight_layout()
plt.show()